In [1]:
import sys
import xarray as xr
import numpy as np
import numpy.ma as ma
import matplotlib.pyplot as plt
import matplotlib.ticker as mticker
import glob
from netCDF4 import Dataset
import cartopy.crs as ccrs
import os

import warnings
warnings.filterwarnings('ignore')

%matplotlib inline
%autosave 60

/scratch/cnt0024/hmg2840/albert7a/anaconda3/lib/python3.7/site-packages/dask/config.py:168: YAMLLoadWarning: calling yaml.load() without Loader=... is deprecated, as the default Loader is unsafe. Please read https://msg.pyyaml.org/load for full details.
  data = yaml.load(f.read()) or {}


Autosaving every 60 seconds


In [2]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/powerspec/powerspec")
import powerspec as pp

In [3]:
sys.path.insert(0,"/scratch/cnt0024/hmg2840/albert7a/DEV/git/diags-CMEMS-on-occigen/common-lib/")
import GriddedData
import WavenumberSpectrum as ws


In [4]:
# Some functions that allow us to compute spectra in boxes

##########################################
def get_values_in_box(imin,imax,jmin,jmax,data):
    values = data[:,jmin:jmax+1,imin:imax+1]
    values = ma.masked_invalid(values)
    return values


##########################################
def compute_spec_for_box(imin,imax,jmin,jmax,data,Mth,navlon,navlat,name,WaveSpecResult,param):
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
    np.savez(WaveSpecResult+'WaveSpec_'+name+'_'+param+'_'+Mth, kspec=kspec ,pspec=pspec)

##########################################
def compute_spec(data,navlon,navlat):
    days = len(data)
    mth_pspec = []
    for it in np.arange(0,days):
        arr = data[it]
        datab = arr.squeeze()
        pspec,kstep = pp.wavenumber_spectra(datab,navlon,navlat)
        mth_pspec.append(pspec)
    mthly_pspec = np.array(mth_pspec)
    mean_mthly_pspec = mthly_pspec.mean(axis=0)
    return kstep,mean_mthly_pspec

In [5]:
box = 'box1'

In [6]:
        imin=626
        imax=1234
        jmin=207
        jmax=976


In [9]:

    param='Vspec'
    varname = 'vomecrty'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    Mth  = '2013m01'
    print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
    filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y'+Mth+'d??.1d_gridVsurf.nc'
    ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
    data = ds[varname]
    if len(ds.nav_lon.dims) == 3:
        navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
    if len(ds.nav_lon.dims) == 2:
        navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
                


/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Vspec/WaveSpec_box1_Vspec_2013m01.npz


In [10]:
pspec.shape

(321,)

In [11]:

    param='Uspec'
    varname = 'vozocrtx'
    WaveSpecResult = '/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-'+str(param)+'/'
    !mkdir -p $WaveSpecResult
    Mth  = '2013m01'
    print(WaveSpecResult+'WaveSpec_'+str(box)+'_'+param+'_'+Mth+'.npz')
    filename='/scratch/cnt0024/hmg2840/albert7a/NATL60/NATL60-CJM165-S/1d/GS/NATL60GS-CJM165_y'+Mth+'d??.1d_gridUsurf.nc'
    ds = xr.open_mfdataset(filename,chunks={'x':500,'y':500,'time_counter':1})
    data = ds[varname]
    if len(ds.nav_lon.dims) == 3:
        navlon=ds.nav_lon[0,imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[0,imin:imax+1,jmin:jmax+1]
    if len(ds.nav_lon.dims) == 2:
        navlon=ds.nav_lon[imin:imax+1,jmin:jmax+1]
        navlat=ds.nav_lat[imin:imax+1,jmin:jmax+1]
    var = get_values_in_box(imin,imax,jmin,jmax,data)
    pspec,kspec = compute_spec(var,navlon,navlat)
                


/scratch/cnt0024/hmg2840/albert7a/NATL60-spec/NATL60GS-Uspec/WaveSpec_box1_Uspec_2013m01.npz


In [12]:
pspec.shape

(321,)